In [1]:
import datetime

In [2]:
begin = datetime.date(2022, 3, 24)
end = datetime.date(2022, 4, 13)
delta = datetime.timedelta(days=1)
split_date = datetime.date(2022, 4, 7)

In [3]:
def generate_scope(d):
    
    cdate = d.strftime("%Y%m%d")
    cdatef = d.strftime("%Y-%m-%d")
    f = open('scopes_3/{}.script'.format(cdate), 'w')

    print(cdate, cdatef)
    cur_scripts = '''//Script 
#DECLARE Origin_data_path string = "/shares/bingads.hm/local/users/Yunfan/samples/{cdate}.tsv";
#DECLARE Origin_header_path string = "/shares/bingads.hm/local/users/Yunfan/samples/header.tsv";
RESOURCE @Origin_header_path;
#DECLARE user_emb_path string = "/shares/bingads.hm/local/AIM/Users/yangxia/UGEEmb/UserEmbedding/IA_TA/{cdatef}_UserEmbedding.ss";
#DECLARE add_emb_path string = "/shares/bingads.hm/local/AIM/Users/yangxia/UGEEmb/AdEmbedding/IA_TA/{cdatef}_AdEmbedding.ss";
#DECLARE Output_path string = "/shares/bingads.hm/local/users/Yunfan/samples_emb/{cdate}.tsv";
#DECLARE OutHeader string = "/shares/bingads.hm/local/users/Yunfan/samples_emb/header_{cdate}.tsv";
#DECLARE ExpiryDuration string = "30";

originData = 
    EXTRACT * 
FROM @Origin_data_path
USING TSVWithHeaderExtractor("-schemaFile", @Origin_header_path);

addkey = SELECT originData.*,
                m_Muid.ToLower() AS m_MuidLower
                FROM originData;

user_emb = SSTREAM @user_emb_path;
add_emb = SSTREAM @add_emb_path;

uadd = SELECT addkey.*
                ,x0 AS Uemb0
                ,x1 AS Uemb1
                ,x2 AS Uemb2
                ,x3 AS Uemb3
                ,x4 AS Uemb4
                ,x5 AS Uemb5
                ,x6 AS Uemb6
                ,x7 AS Uemb7
                ,x8 AS Uemb8
                ,x9 AS Uemb9
                ,x10 AS Uemb10
                ,x11 AS Uemb11
                ,x12 AS Uemb12
                ,x13 AS Uemb13
                ,x14 AS Uemb14
                ,x15 AS Uemb15
                ,x16 AS Uemb16
                ,x17 AS Uemb17
                ,x18 AS Uemb18
                ,x19 AS Uemb19
                ,x20 AS Uemb20
                ,x21 AS Uemb21
                ,x22 AS Uemb22
                ,x23 AS Uemb23
                ,x24 AS Uemb24
                ,x25 AS Uemb25
                ,x26 AS Uemb26
                ,x27 AS Uemb27
                ,x28 AS Uemb28
                ,x29 AS Uemb29
                ,x30 AS Uemb30
                ,x31 AS Uemb31
        FROM addkey INNER JOIN user_emb
        ON addkey.m_MuidLower == user_emb.UserId;

output = SELECT uadd.*
                ,x0 AS Aemb0
                ,x1 AS Aemb1
                ,x2 AS Aemb2
                ,x3 AS Aemb3
                ,x4 AS Aemb4
                ,x5 AS Aemb5
                ,x6 AS Aemb6
                ,x7 AS Aemb7
                ,x8 AS Aemb8
                ,x9 AS Aemb9
                ,x10 AS Aemb10
                ,x11 AS Aemb11
                ,x12 AS Aemb12
                ,x13 AS Aemb13
                ,x14 AS Aemb14
                ,x15 AS Aemb15
                ,x16 AS Aemb16
                ,x17 AS Aemb17
                ,x18 AS Aemb18
                ,x19 AS Aemb19
                ,x20 AS Aemb20
                ,x21 AS Aemb21
                ,x22 AS Aemb22
                ,x23 AS Aemb23
                ,x24 AS Aemb24
                ,x25 AS Aemb25
                ,x26 AS Aemb26
                ,x27 AS Aemb27
                ,x28 AS Aemb28
                ,x29 AS Aemb29
                ,x30 AS Aemb30
                ,x31 AS Aemb31
        FROM uadd INNER JOIN add_emb
        ON uadd.m_AdId == add_emb.AdId;

OUTPUT output
TO @Output_path
    WITH STREAMEXPIRY @ExpiryDuration;

SELECT TOP 1 *;
OUTPUT TO @OutHeader WITH STREAMEXPIRY @ExpiryDuration USING HeaderOutputter ;  

'''.format(cdate=cdate, cdatef=cdatef) + \
    '''
#CS
using System;
using System.Collections.Generic;
using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Linq;
using ScopeRuntime;


public class RandomUtils
{
    /// <summary>
    /// Get a random UInt64
    /// </summary>
    /// <returns>Random UInt64</returns>
    public static double GetRandomConsistentDoubleFromRGUID(string RGUID)
    {
        double rn = 0.99;
        Guid gd = new Guid();

        try
        {
            gd = Guid.Parse(RGUID);
        }
        catch (Exception e)
        {
            return rn;
        }

        byte[] bytes = gd.ToByteArray();
        int seed = BitConverter.ToInt32(bytes, 0);
        double rp = new Random(seed).NextDouble();

        return rp;
    }


}


public class TSVWithHeaderExtractor : Extractor
{
    private bool initialized = false;
    private string delim = "\t";
    private string schemaFile = "";
    private string schemaString = "";

       public static string LegalizeColumnName(string name)
    {
        string legalized = name.Replace("DateTime", "DateTime_");
        legalized = Regex.Replace(legalized, @"(^m:)", "m_");
        legalized = Regex.Replace(legalized, @"(@)", "_atsymbol_");

        // replace the illegal char brought in by Category transform and Generic transform
        legalized = Regex.Replace(legalized, @"([^\w\d_]+)", "_");

        // and cut the length to 450 if it's too long. C# has a 512 char length limit to identifier(so does scope)
        legalized = legalized.Substring(0, Math.Min(450, legalized.Length)); // cap length to 450

        return legalized;
    }

    public void Initialize(string[] args)
    {
        if (initialized)
            return;

        // create and add 
        // Process args, if any
        for (Int32 i = 0; i < args.Length; i++)
        {
            switch (args[i])
            {
                case "-schemaFile":
                    if (i == args.Length - 1)
                        throw new ArgumentException("No value specified", "schemaFile");
                    schemaFile = Path.GetFileName(args[++i]); //only get file name

                    // load up schemafile
                    using (StreamReader sr = new StreamReader(schemaFile))//read header
                    {
                        schemaString = string.Join(",", sr.ReadLine().Trim().Split('\t').Select(t => LegalizeColumnName(t)).ToArray());
                    }
                    break;

                case "-delim":
                    if (i == args.Length - 1)
                        throw new ArgumentException("No value specified", "delim");
                    delim = args[++i];
                    break;
            }
        }

        initialized = true;
    }

    public override Schema GetOutputSchemaAtCompileTime(string[] requestedColumns, string[] args)
    {
        Initialize(args);
        return new Schema(schemaString);
    }

    public override IEnumerable<Row> Extract(StreamReader reader, Row outputRow, string[] args)
    {
        Initialize(args);

        string line;
        int count = 0;

        while ((line = reader.ReadLine()) != null)
        {
            count++;

            if (count == 1 && (line.StartsWith("m:", StringComparison.OrdinalIgnoreCase) || line.StartsWith("Feature_", StringComparison.OrdinalIgnoreCase)))//skip header
            {
                continue;
            }

            string[] tokens = line.Split(delim.ToCharArray());
            if (tokens.Length < outputRow.Count)
            {
                continue;
            }

            try
            {
                for (int i = 0; i < tokens.Length; ++i)
                {
                    outputRow[i].UnsafeSet(tokens[i]);
                }
            }
            catch(Exception)
            {
                continue;
            }

            yield return outputRow;
        }
    }
}
public class HeaderOutputter : Outputter
{
       public static string RevertLegalizeColumnName(string name)
    {
        string legalized = name.Replace("DateTime_", "DateTime");
        legalized = Regex.Replace(legalized, @"(^m_)", "m:");
        legalized = Regex.Replace(legalized, @"(_atsymbol_)", "@");
        return legalized;
    }

       public override void Output(RowSet input, StreamWriter writer, string[] args)
       {
              // output the header
              writer.WriteLine(string.Join("\t", input.Schema.ToString().Split(',').Select(c => RevertLegalizeColumnName(c.Split(':')[0])).ToArray()));
              writer.Flush();
       }
}
#ENDCS
'''
    f.write(cur_scripts)
    f.close()

In [4]:
d = begin
while d <= end:
    next_date = d + delta
    if d < split_date:
        generate_scope(d)
    else:
        generate_scope(d)
    d = next_date

20220324 2022-03-24
20220325 2022-03-25
20220326 2022-03-26
20220327 2022-03-27
20220328 2022-03-28
20220329 2022-03-29
20220330 2022-03-30
20220331 2022-03-31
20220401 2022-04-01
20220402 2022-04-02
20220403 2022-04-03
20220404 2022-04-04
20220405 2022-04-05
20220406 2022-04-06
20220407 2022-04-07
20220408 2022-04-08
20220409 2022-04-09
20220410 2022-04-10
20220411 2022-04-11
20220412 2022-04-12
20220413 2022-04-13
